## Wczytanie bibliotek

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime as dt
import scipy.signal as ss
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
import group_lasso
from sklearn.metrics import mean_squared_error
import os

## Ścieżka z modelami

In [58]:
ROOT = "C:/Users/ndzad/Dropbox/MINI/WTUM/Modele/"

## Wczytanie zbiorów

In [194]:
holidays_events = pd.read_csv("https://www.dropbox.com/s/bxyamlpevkiwwoq/holidays_events.csv?dl=1")
holidays_events["holiday_type"] = holidays_events["type"]
holidays_events.drop(["type"],axis=1,inplace=True)
oil = pd.read_csv("https://www.dropbox.com/s/l6ln0ztl4m0pw3a/oil.csv?dl=1",parse_dates=['date'],index_col='date')
oil2 = pd.read_csv("https://www.dropbox.com/s/l6ln0ztl4m0pw3a/oil.csv?dl=1")
sample_submission = pd.read_csv("https://www.dropbox.com/s/68jjl61x6u3klos/sample_submission.csv?dl=1")
stores = pd.read_csv("https://www.dropbox.com/s/lcxn6r9bs2exguq/stores.csv?dl=1")
test = pd.read_csv("https://www.dropbox.com/s/cvdo1gn7r5lu2uz/test.csv?dl=1",index_col='id')
train = pd.read_csv("https://www.dropbox.com/s/s8p2b5awnuqfk0d/train.csv?dl=1",index_col='id')
transactions = pd.read_csv("https://www.dropbox.com/s/92fij9bcwt0e0cj/transactions.csv?dl=1")

In [71]:
import warnings
warnings.filterwarnings("ignore")

## Funkcje pomocnicze

Funkcja przygotowująca dane objaśniające. Łączy zbiory i zmienia zmienne kategoryczne na zmienne numeryczna metodą One Hot Encoding, a także grupuje po dniu lub miesiącu.

In [163]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [190]:
def substraction(lst1, lst2):
    lst3 = [value for value in lst1 if value not in lst2]
    return lst3

In [191]:
def przygotowanie_danych(df: pd.DataFrame, type, type_item):
    """
    Funkcja łącząca zbiory i zmieniająca zmienne kategoryczne na zmienne numeryczne. Funkcja jest przewidziana
    df - zbiór danych treningowych
    type - dane sklepowe ('family') / dane rodzinne ('store_nbr')
    type_item - konretny element zmiennej type np. dla type=="family" type_item=="AUTOMOTIVE"
    """
    #Łaczymy z pozostałymi zbiorami

    #Zbiór stores
    df = df.merge(stores,how="left",left_on=['store_nbr'],right_on=['store_nbr'])

    #Zbiór transactions
    df = df.merge(transactions,how="left",left_on=['date','store_nbr'],right_on=['date','store_nbr'])

    #Zbiór holidays_events
    df = df.merge(holidays_events,how="left",left_on=['date'],right_on=['date'])

    #Dodanie oil
    df = df.merge(oil2,how="left",left_on=['date'],right_on=['date'])

    #Interpolacja braków danych z oil
    df.interpolate(method ='linear', limit_direction ='backward', inplace=True)

    #Wybieramy family ze zbioru traningowego
    df_fam = df.loc[(df[type]==type_item)]
    
    #Dodajemy zmienne na dzień tygodnia i na miesiąc
    df_fam['dayofweek'] = pd.DatetimeIndex(df_fam['date']).dayofweek + 1
    df_fam['month'] = pd.to_datetime(df_fam['date']).dt.month

    #Usunięcie zmiennych, które nie będą zmieniane
    df_fam.drop([type,"description","transferred"],axis=1,inplace=True)
    if type == "family":
        type_opposite = "store_nbr"
    elif type == "store_nbr":
        type_opposite = "family"

    #One Hot Encoding
    df_fam = pd.get_dummies(df_fam,columns=["locale",type_opposite,"city", "state", "type", "cluster","locale_name", "holiday_type"],prefix=["locale",type_opposite,"city", "state", "type", "cluster","locale_name", "holiday_type"])
    
    #W zbiorze test jest tylko jeden miesiąc dlatego musimy zrobić technicnzy zabieg polegający na dodaniu kolumn z samymi zerami.
    if 'sales' not in df_fam.columns:        
        missing_cols = substraction(przygotowanie_danych(train,type,type_item).columns,df_fam.columns)
        for i in missing_cols:
            df_fam[i] = 0

    return(df_fam)

In [4]:
def dane_drzewo(df: pd.DataFrame, type, type_item):
    df2 =df.copy()
    df2["family"], uniques=pd.factorize(df2["family"])
    df2["city"], uniques=pd.factorize(df2["city"])
    df2["state"], uniques=pd.factorize(df2["state"])
    df2["type"], uniques=pd.factorize(df2["type"])
    df2["locale"], uniques=pd.factorize(df2["locale"])
    df2["locale_name"], uniques=pd.factorize(df2["locale_name"])
    df2["holiday_type"], uniques=pd.factorize(df2["holiday_type"])
    df2.fillna(0,inplace=True)

Podział danych na część treningową i walidacyjną.

In [5]:
def podzial(df: pd.DataFrame, date='2016-06-01'):
    """
    Funkcja dzieli zbiór df na cześć treningową i walidacyjną względem date.
    """
    train = df.loc[(df['date']<date)].drop(["date"],axis=1)
    test = df.loc[(df['date']>=date)].reset_index(drop=True).drop(["date"],axis=1)
    return(train,test)

## Funkcje modelujące

Model XGBoost

In [73]:
def model_xgb(df1: pd.DataFrame,df2: pd.DataFrame, model_name: str):
    """
    Funkcja tworzy model XGBoost na danych df1 i waliduje na danych df2.
    """
    #Zbiory danych treningowy
    X = df1.drop(['sales'],axis=1).copy()
    Y = df1['sales'].copy()

    #Zbiory danych walidacyjnych 
    X_val = df2.drop(['sales'],axis=1)
    Y_val = df2['sales'].copy()

    #Dopasowanie modelu
    model = xgb.XGBRegressor()
    model.fit(X,Y)
    Y_pred = model.predict(X_val)

    #RMSE - jakość dopasowania
    RMSE = mean_squared_error(Y_val,Y_pred,squared=False)/(np.mean(Y_val))
    print("Model: ", model_name, " RMSE: ",RMSE)

    sciezka = ROOT + model_name + ".txt"
    model.save_model(sciezka)
    return(RMSE)

Model regresji liniowej

In [7]:
def model_ols(df1: pd.DataFrame,df2: pd.DataFrame, model_name: str):
    """
    Funkcja tworzy model regresji liniowej metodą najmniejszych kwadratów na danych df1 i waliduje na danych df2.
    """
    #Zbiory danych treningowy
    X = df1.drop(['sales'],axis=1).copy()
    Y = df1['sales'].copy()

    #Zbiory danych walidacyjnych 
    X_val = df2.drop(['sales'],axis=1)
    Y_val = df2['sales'].copy()

    #Dopasowanie modelu
    model = sm.OLS(Y,X).fit()
    Y_pred = model.predict(X_val)

    #RMSE - jakość dopasowania
    RMSE = mean_squared_error(Y_val,Y_pred,squared=False)/(np.mean(Y_val))
    print("Model: ", model_name, " RMSE: ",RMSE)

    sciezka = ROOT + model_name + ".txt"
    model.save_model(sciezka)
    return(RMSE)

Model drzewa regresyjnego

In [8]:
def model_tree(df1: pd.DataFrame,df2: pd.DataFrame, model_name: str):
    """
    Funkcja tworzy model drzewa regresyjnego na danych df1 i waliduje na danych df2.
    """
    #Zbiory danych treningowy
    X = df1.drop(['sales'],axis=1).copy()
    Y = df1['sales'].copy()

    #Zbiory danych walidacyjnych 
    X_val = df2.drop(['sales'],axis=1)
    Y_val = df2['sales'].copy()

    #Dopasowanie modelu
    model = DecisionTreeRegressor().fit(X,Y)
    Y_pred = model.predict(X_val)

    #RMSE - jakość dopasowania
    RMSE = mean_squared_error(Y_val,Y_pred,squared=False)/(np.mean(Y_val))
    print("Model: ", model_name, " RMSE: ",RMSE)

    sciezka = ROOT + model_name + ".txt"
    model.save_model(sciezka)
    return(RMSE)

In [16]:
def model_group_lasso(df1,df2, model_name: str):
    """
    Funkcja tworzy model grupowego lasso na danych df1 i waliduje na danych df2.
    """
    #Zbiory danych treningowy
    X = df1.drop(['sales'],axis=1).copy()
    Y = df1['sales'].copy()

    #Zbiory danych walidacyjnych 
    X_val = df2.drop(['sales'],axis=1)
    Y_val = df2['sales'].copy()

    #Dopasowanie modelu
    model = group_lasso.GroupLasso(groups=[-1,-1, 1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,-1]).fit(X,Y)
    Y_pred = model.predict(X_val)

    #RMSE - jakość dopasowania
    RMSE = mean_squared_error(Y_val,Y_pred,squared=False)/(np.mean(Y_val))
    print("Model: ", model_name, " RMSE: ",RMSE)

    sciezka = ROOT + model_name + ".txt"
    model.save_model(sciezka)
    return(RMSE)

## Funckja do predykcji

In [150]:
def predykcja (df: pd.DataFrame, model: xgb.XGBRegressor, folder: str):
    """
    Funkcja wykonuje predykcje modelu model na zbiorze df i odpisuje w formacie .csv do folderu folder.
    """
    mod2 = xgb.XGBRegressor()
    mod2.load_model(ROOT+model+".txt")
    df_res =df.copy()
    pred = mod2.predict(df)
    df_res["sales_pred"] = pred
    pred_csv = df_res.to_csv(folder+model+".csv",index=False)

    return(pred_csv)

## Modelowanie

In [72]:
(train2,test2) = podzial(przygotowanie_danych(train,'family','AUTOMOTIVE'),'2016-06-01')

In [67]:
RMSE_xgb = model_xgb(train2,test2,"test")

0.2647430624584262


In [139]:
(train3,test3) = podzial(przygotowanie_danych(train,'store_nbr',1),'2016-06-01')

In [30]:
(model_xgb, RMSE_xgb) = model_xgb(train3,test3)

0.6198961685176666


## Model dla każdej rodziny. Modele odpisywane są do ROOT - ścieżki podanej na początku.

# Modelowanie

In [75]:
families = np.array(train["family"].unique())
family_RMSEs = []

In [76]:
for i in families:
    (train2,test2) = podzial(przygotowanie_danych(train,'family',i),'2016-06-01')
    RMSE_xgb = model_xgb(train2,test2,i)
    family_RMSEs.append(RMSE_xgb)

Model:  AUTOMOTIVE  RMSE:  0.6861653699517859
Model:  BABY CARE  RMSE:  3.4948295658920756
Model:  BEAUTY  RMSE:  0.7205881764239868
Model:  BEVERAGES  RMSE:  0.31779410053589807
Model:  BOOKS  RMSE:  3.994430513647246
Model:  BREAD_BAKERY  RMSE:  0.2647430624584262
Model:  CELEBRATION  RMSE:  1.170489191002666
Model:  CLEANING  RMSE:  0.30010944377920146
Model:  DAIRY  RMSE:  0.22848451644533507
Model:  DELI  RMSE:  0.25711408962040094
Model:  EGGS  RMSE:  0.5358559262506273
Model:  FROZEN FOODS  RMSE:  1.183994333148742
Model:  GROCERY I  RMSE:  0.34631785551534033
Model:  GROCERY II  RMSE:  0.8921728224904292
Model:  HARDWARE  RMSE:  1.2474795856009246
Model:  HOME AND KITCHEN I  RMSE:  1.2312579643318984
Model:  HOME AND KITCHEN II  RMSE:  1.395166723816384
Model:  HOME APPLIANCES  RMSE:  2.027397594974563
Model:  HOME CARE  RMSE:  0.3832039660347386
Model:  LADIESWEAR  RMSE:  1.0003645281008315
Model:  LAWN AND GARDEN  RMSE:  1.5703395588668807
Model:  LINGERIE  RMSE:  1.611733919

# Predykcja

In [187]:
time = dt.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")
folder2 = ROOT+"predict_family_"+time+"/"
os.makedirs(folder2)

In [193]:
mod2 = xgb.XGBRegressor()
for i in families:
    t = przygotowanie_danych(test,"family",i)
    t.drop(["date","sales"],axis=1,inplace=True)
    mod2.load_model(ROOT+i+".txt")
    mod2.predict(test2.drop(["sales"],axis=1))
    predykcja(t,i,folder2)

ValueError: Wrong number of items passed 0, placement implies 1

## Model dla każdego sklepu. Modele odpisywane są do ROOT - ścieżki podanej na początku.

# Modelowanie

In [179]:
stores_nbr = np.array(train["store_nbr"].unique())
stores_nbr.sort()
stores_nbr_RMSEs = []

In [183]:
for i in stores_nbr:
    (train3,test3) = podzial(przygotowanie_danych(train,'store_nbr',i),'2016-06-01')
    RMSE_xgb = model_xgb(train3,test3,str(i))
    stores_nbr_RMSEs.append(RMSE_xgb)

Model:  1  RMSE:  0.6198961685176666
Model:  10  RMSE:  0.6463020048718221
Model:  11  RMSE:  0.5876590283466362
Model:  12  RMSE:  0.6673139636253224
Model:  13  RMSE:  0.6578999813581597
Model:  14  RMSE:  0.8180761879968397
Model:  15  RMSE:  0.6871555087511951
Model:  16  RMSE:  0.9739440071073853
Model:  17  RMSE:  0.6157637579617721
Model:  18  RMSE:  1.7489272545172623
Model:  19  RMSE:  0.587862395866956
Model:  2  RMSE:  1.2022746613558344
Model:  20  RMSE:  2.65075713710132
Model:  21  RMSE:  0.6844599029862802
Model:  22  RMSE:  0.716448652343852
Model:  23  RMSE:  0.5712708856222248
Model:  24  RMSE:  0.7491045307930392
Model:  25  RMSE:  1.6216000322047095
Model:  26  RMSE:  1.3043151073088282
Model:  27  RMSE:  0.6134293749978931
Model:  28  RMSE:  0.5385021907096724
Model:  29  RMSE:  0.5810905996616288
Model:  3  RMSE:  0.6386473097009742
Model:  30  RMSE:  1.0009509955777756
Model:  31  RMSE:  0.6092144973726266
Model:  32  RMSE:  2.292927827340676
Model:  33  RMSE:  0

# Predykcja

In [184]:
t = przygotowanie_danych(test,"family","AUTOMOTIVE")
t.drop(["date","sales"],axis=1,inplace=True)
time = dt.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")
folder2 = ROOT+"predict_store_nbr_"+time+"/"
os.makedirs(folder2)

In [185]:
mod2 = xgb.XGBRegressor()
for i in stores_nbr:
    mod2.load_model(ROOT+i+".txt")
    mod2.predict(test2.drop(["sales"],axis=1))
    predykcja(t,i,folder2)

- Outliery (zrobione)
- Grupowanie po family,typie sklep (uśrednienie sprzedaży po dacie)
- Agregacja i predykcja na miesiąc 